# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [217]:
import re
from collections import Counter
import pandas as pd

In [218]:
WORDS = set() #dictionary of words

#read bigrams
bigram = dict()
with open('bigrams.txt', mode = 'rb') as f:
  content = f.read()
  s = str(content)
  w = s[2:-1].replace('\\t',' ').replace('\\r\\n',' ').split()
  for i in range(0, len(w), 3):
    if not (w[i+1].lower() in bigram):
      bigram[w[i+1].lower()] = dict()
    bigram[w[i+1].lower()][w[i+2].lower()] = int(w[i])
    WORDS.add(w[i+1].lower())
    WORDS.add(w[i+2].lower())


In [219]:
#count all words
all_words = 0
for w in bigram:
  all_words += sum(bigram[w].values())
all_words

286758206

In [220]:
def pos(ps):
  rez = ps[:2]
  return rez

In [221]:
LINKS = dict()
with open('coca_all_links.txt', mode = 'rb') as f:
  content = f.read()
  w = str(content)
  w = w[2:-5].split('\\r\\n')
  w[78801] = '28\\thuman\\tsinfulness\\tjj\\tnn1' # it was broken inside doc:(
  w[117501]= '50\\trural\\tenvironment\\tjj\\tnn1'
  for l in w:
    spl = l.split('\\t')
    if (len(spl) != 5):
      continue
    count = int(spl[0])
    word1 = spl[1].lower()
    word2 = spl[2].lower()
    prt1 = pos(spl[3])
    prt2 = pos(spl[4])
    union1 = (word1, prt1)
    union2 = (word2, prt2)
    if not(union1 in LINKS):
      LINKS[union1] = dict()
    LINKS[union1][union2] = count

In [222]:
def known(words):
    #subset of subwords from WORDS dictionary
    return set(w for w in words if w in WORDS)

In [223]:
def edits1(word):
    #All edits that are one edit away from `word`.
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    #All edits that are two edits away from `word`.
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1))

In [224]:
# function for fixing misclicks
neighbor = {'a': ['q', 's', 'w', 'z'],
            'b': ['g', 'h', 'n', 'v'],
            'c': ['d', 'f', 'v', 'x'],
            'd': ['c', 'e', 'f', 'r', 's', 'x'],
            'e': ['d', 'r', 's', 'w'],
            'f': ['c', 'd', 'g', 'r', 't', 'v'],
            'g': ['b', 'f', 'h', 't', 'v', 'y'],
            'h': ['b', 'g', 'j', 'n', 'u', 'y'],
            'i': ['j', 'k', 'o', 'u'],
            'j': ['h', 'i', 'k', 'm', 'n', 'u'],
            'k': ['i', 'j', 'l', 'm', 'o'],
            'l': ['k', 'o', 'p'],
            'm': ['j', 'k', 'n'],
            'n': ['b', 'h', 'j', 'm'],
            'o': ['i', 'k', 'l', 'p'],
            'p': ['l', 'o'],
            'q': ['a', 'w'],
            'r': ['d', 'e', 'f', 't'],
            's': ['a', 'd', 'e', 'w', 'x', 'z'],
            't': ['f', 'g', 'r', 'y'],
            'u': ['h', 'i', 'j', 'y'],
            'v': ['b', 'c', 'f', 'g'],
            'w': ['a', 'e', 'q', 's'],
            'x': ['c', 'd', 's', 'z'],
            'y': ['g', 'h', 't', 'u'],
            'z': ['a', 's', 'x'],
            '1': ['q'],
            '2': ['q', 'w'],
            '3': ['e', 'w'],
            '4': ['e', 'r'],
            '5': ['r', 't'],
            '6': ['t', 'y'],
            '7': ['u', 'y'],
            '8': ['i', 'u'],
            '9': ['i', 'o'],
            '0': ['o', 'p'],
            '-': ['p'],
            '.': ['l'],
            ',': ['l', 'm'],
            '[': ['p'],
            ';': ['l', 'p'],
}
def edit_mis(word):
  splits = [(word[:i], word[i:])    for i in range(len(word) + 1)]
  replaces = [L + c + R[1:] for L, R in splits if R for c in neighbor[R[0]]]
  return known(replaces)

In [225]:
def count_bi_prob(first_w, second_w):
  if not (first_w in bigram and second_w in bigram[first_w]):
    return 1.0e-12
  count = bigram[first_w][second_w]
  N = sum(bigram[first_w].values())
  prob = count / N
  return prob

In [226]:
def cmp(word):
  if not(word in bigram): return 0
  return sum(bigram[word].values())/all_words

In [266]:
def best_variant(word_list, prev_word=None, next_word=None, k=-1):
  if (k == -1): k = len(word_list)
  counts = []
  if (prev_word == None and next_word == None):
    counts = [(cmp(word), word) for word in word_list]
  elif (prev_word == None):
    for word in word_list:
      prob = count_bi_prob(word, next_word)
      if (prob > 1e-24):
        counts.append((prob, word))
  elif (next_word == None):
    for word in word_list:
      prob = count_bi_prob(prev_word, word)
      if (prob > 1e-24):
        counts.append((prob, word))
  else:
    for word in word_list:
      prob = count_bi_prob(prev_word, word)*count_bi_prob(word, next_word)
      if (prob > 1e-24):
        counts.append((prob, word))
  counts.sort(reverse=True)
  return set(counts[:k])

In [238]:
import spacy
nlp = spacy.load('en_core_web_sm')

def link_pos(part): #this is because diferences in notation
  if (part[:3] == 'ADJ'):
    return 'jj'
  elif (part[:4] == 'VERB'):
    return 'vv'
  elif (part[:4] == 'NOUN'):
    return 'nn'
  else:
    return 'th' #other parts of speach

In [229]:
def concate_string(start, word, end):
  rez = ''
  for w in start:
    rez = rez + w + ' '
  rez = rez + word
  if (len(end) == 0): return rez
  for w in end:
    rez = rez + ' ' + w
  return rez

In [230]:
def count_link_prob(first_w, second_w):
  if not (first_w in LINKS and second_w in LINKS[first_w]):
    return 1.0e-12
  count = LINKS[first_w][second_w]
  N = sum(LINKS[first_w].values())
  prob = count / N
  return prob

In [300]:
def choose_word(weighted, start, end, k=3):
  result_prob = []
  for word in weighted:
    prob = weighted[word]

    words_pos = []
    doc = nlp(concate_string(start, word, end))
    for token in doc:
      words_pos.append((token.text, link_pos(token.pos_)))
      if (token.text == word):
        unit_w = (token.text, link_pos(token.pos_))

    st = True
    for unit in words_pos:
      if (unit == unit_w):
        st = False
        continue
      if (st):
        prob = prob * count_link_prob(unit, unit_w)
      else:
        prob = prob * count_link_prob(unit_w, unit)

    result_prob.append((prob, word))
  result_prob.sort(reverse=True)
  return result_prob[:k]

In [335]:
def correct_sentence(sentence, poss_k=3):
  words = sentence.lower().split()
  k = known(words)
  if len(k) == len(words):
    return concate_string([],words[0],words[1:])
  new_sentence = ''
  for i in range(len(words)):
    if (words[i] in k):
      new_sentence += words[i] + ' '
      continue
    #find unknown word
    word = words[i]
    prev_word = None if i == 0 else words[i-1]
    next_word = None if i == len(words)-1 else words[i+1]

    misses = best_variant(edit_mis(word), prev_word, next_word)
    edit1 = best_variant(edits1(word), prev_word, next_word)
    edit2 = best_variant(edits2(word), prev_word, next_word)

    weighted = dict()
    for (c, word) in misses:
      weighted[word] = c*0.9
    for (c, word) in edit1:
      if not(word in weighted):
        weighted[word] = c*0.8
    for (c, word) in edit2:
      if not(word in weighted):
        weighted[word] = c*0.16

    possible_replaces = choose_word(weighted, words[:i], words[i+1:],poss_k)
    if (len(possible_replaces) > 0):
      new_word = possible_replaces[0][1]
    else:
      new_word = word
      WORDS.add(word) # if it complitly new word (assume it has no mistakes)
    new_sentence += new_word + ' '
    if (poss_k != 1):
      print(f"Possible replaces:")
      for i in range(len(possible_replaces)):
        print(f"  {possible_replaces[i][1]} ({possible_replaces[i][0]})")
  return new_sentence

In [333]:
correct_sentence('I am dking sport')

Possible replaces:
  doing (5.49719297400188e-51)
  going (4.50666216899983e-51)
  dying (7.035635978122063e-52)


'i am doing sport '

In [327]:
correct_sentence("The wall was on my way", 1)

'the wall was on my way'

## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

The fivegrams is better then bigrams on long sentences, but for this tas long sentences are less likely than short ones (like searching queries). In my implementation, weights for `edit1` and `edit2` are `0.8` and `0.16`, because I consider that probability of two mistakes are twice less that probapility of one mistake. Also in my implementation I take miskliks in attention in `edit_mis` and its have higher probability then `edit1`.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

In [ ]:
test_sentenses = [
    'This is correct sentence',
    'This is incoffect sentence',
    'Just sent3nse',
    'This is the emd of testinh'
]

for i in range(len(test_sentenses)):
  print(f"{i}. {test_sentenses[i]}")
  correct_sentence(test_sentenses[i])

0. This is correct sentence
1. This is incoffect sentence
Possible replaces:
  incorrect (1.1097224350621847e-53)
2. Just sent3nse
